# テスト用ノートブック

### 参考資料
https://qiita.com/tatsuki-tsuchiyama/items/4c996ce740cc0c607d35

In [1]:
import configparser
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase

In [ ]:
from langchain import SQLDatabaseChain
#from langchain_experimental.sql import SQLDatabaseChain

In [4]:
# 設定ファイルの読み込み
config = configparser.ConfigParser() #Configのハンドル設定
config.read("./private/config.ini")

#OpenAI部分の設定参照
openai_config = config["OPENAI"] 
input_openai_api_key = openai_config["OPENAI_API_KEY"]

In [5]:
#データベースとモデルの設定
db = SQLDatabase.from_uri("sqlite:///./db/chinook.db")

In [6]:
llm = ChatOpenAI(
    openai_api_key=input_openai_api_key,
    temperature=0,
    model_name="gpt-3.5-turbo",
    streaming=True,
)

In [12]:
agent_executor = SQLDatabaseChain(
    llm=llm,
    database=db,
    verbose=True
)

In [13]:
agent_executor.run("いくつのデータが存在しますか？")



> Entering new SQLDatabaseChain chain...
いくつのデータが存在しますか？
SQLQuery:

c:\01_program\01_github\101_004_Webapp_SQLRAG\venv\llm_SQL_app\Lib\site-packages\langchain\sql_database.py:191: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows = connection.execute(command)


SELECT COUNT(*) AS TotalData FROM artists;
SQLResult: [(275,)]
Answer:275データが存在します。
> Finished chain.


'275データが存在します。'

In [14]:
agent_executor.run("データ項目を教えて")



> Entering new SQLDatabaseChain chain...
データ項目を教えて
SQLQuery:SELECT name
FROM sqlite_master
WHERE type = 'table'
LIMIT 5;
SQLResult: [('albums',), ('sqlite_sequence',), ('artists',), ('customers',), ('employees',)]
Answer:albums, sqlite_sequence, artists, customers, employees
> Finished chain.


'albums, sqlite_sequence, artists, customers, employees'

In [16]:
agent_executor.run("playlistedのデータを見せて")



> Entering new SQLDatabaseChain chain...
playlistedのデータを見せて
SQLQuery:SELECT "PlaylistId", "TrackId"
FROM playlist_track
LIMIT 5;

SQLResult: [(1, 3402), (1, 3389), (1, 3390), (1, 3391), (1, 3392)]
Answer:Playlistedのデータは以下の通りです:
(1, 3402)
(1, 3389)
(1, 3390)
(1, 3391)
(1, 3392)
> Finished chain.


'Playlistedのデータは以下の通りです:\n(1, 3402)\n(1, 3389)\n(1, 3390)\n(1, 3391)\n(1, 3392)'